This notebook demonstrates the use of the `FilterWheel` in Scopesim. The METIS configuration contains two instances of this effect, named `filter_wheel` (for science filters) and `nd_filter_wheel` (for neutral-density filters). Each filter wheel contains a number of predefined filters, with different filter sets for the LM- and N-band imagers. 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

In [ ]:
import scopesim as sim
sim.bug_report()

# Edit this path if you have a custom install directory, otherwise comment it out.
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../../../"

If you haven't got the instrument packages yet, uncomment the following cell.

In [ ]:
# sim.download_package(["instruments/METIS", "telescopes/ELT", "locations/Armazones"])

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['img_lm'])

The filter to use is defined by setting `!OBS.filter_name`. In `img_lm` mode, it defaults to the Lp filter:

In [ ]:
cmd['!OBS.filter_name']

In [ ]:
metis = sim.OpticalTrain(cmd)

The METIS package defines the list of filters that are available in the real instrument:

In [ ]:
metis['filter_wheel'].filters

At any moment one of these filters is in the optical path and used for the simulation. Initially, this is the one set by `!OBS.filter_name`:

In [ ]:
metis['filter_wheel'].current_filter

The current filter can be changed to any of the filters in the list:

In [ ]:
metis['filter_wheel'].change_filter("PAH_3.3")

In [ ]:
metis['filter_wheel'].current_filter

# Observing the same source in different filters

In [ ]:
src = sim.source.source_templates.empty_sky()

In [ ]:
metis['filter_wheel'].change_filter("Lp")

metis.observe(src)
img_Lp = metis.image_planes[0].data

In [ ]:
metis['filter_wheel'].change_filter("PAH_3.3")

metis.observe(src, update=True)
img_PAH = metis.image_planes[0].data

In [ ]:
print("Background in Lp:      {:8.1f} counts/s".format(np.median(img_Lp)))
print("Background in PAH_3.3: {:8.1f} counts/s".format(np.median(img_PAH)))

# Using the neutral-density filter wheel

METIS also has neutral-density filters that can be inserted and changed using the `nd_filter_wheel` effect. The transmission of the filter `ND_ODx` is $10^{-x}$.

In [ ]:
metis['nd_filter_wheel'].filters

In [ ]:
metis['nd_filter_wheel'].current_filter

Observe a bright star (default arguments result in Vega at 0 mag) in the Lp filter. It will be found that the star saturates the detector in the open position, and requires the `ND_OD4` filter not to do so.

In [ ]:
star = sim.source.source_templates.star()

In [ ]:
metis['filter_wheel'].change_filter('Lp')

In [ ]:
metis['nd_filter_wheel'].change_filter("open")
metis.observe(star, update=True)
hdu_open = metis.readout()[0][1]

In [ ]:
metis['nd_filter_wheel'].change_filter("ND_OD3")
metis.observe(star, update=True)
hdu_OD3 = metis.readout()[0][1]

In [ ]:
metis['nd_filter_wheel'].change_filter("ND_OD4")
metis.observe(star, update=True)
hdu_OD4 = metis.readout()[0][1]

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(131)
plt.imshow(hdu_open.data[700:1350, 700:1350], origin='lower', norm=LogNorm(vmin=1e-3, vmax=2e6))
plt.colorbar()
plt.subplot(132)
plt.imshow(hdu_OD3.data[700:1350, 700:1350], origin='lower', norm=LogNorm(vmin=1e-3, vmax=2e6))
plt.colorbar()
plt.subplot(133)
plt.imshow(hdu_OD4.data[700:1350, 700:1350], origin='lower', norm=LogNorm(vmin=1e-3, vmax=2e6))
plt.colorbar()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(15, 4))

ax1.plot(hdu_open.data[800:1250, 1024])
ax1.set_title("ND filter: open")
ax2.plot(hdu_OD3.data[800:1250, 1024])
ax2.set_title("ND filter: 1e-3")
ax3.plot(hdu_OD4.data[800:1250, 1024])
ax3.set_title("ND filter: 1e-4")